In [2]:
# simple setting for background

%matplotlib inline
import numpy as np
import sklearn.linear_model as linear_model
import seaborn as sns
import xgboost as xgb
from sklearn.model_selection import KFold
from IPython.display import HTML, display
from sklearn.manifold import TSNE
from sklearn.cluster import KMeans
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
import tensorflow as tf
import sklearn
import matplotlib.pyplot as plt
import pandas as pd
import warnings # Hide the warning for sns.distplot
warnings.filterwarnings('ignore') # Not suggest since it may ignore critical warning

Init Plugin
Init Graph Optimizer
Init Kernel


In [50]:
# Data Loading
df_annual_sales = pd.read_csv('Data/AnnualTicketSales.csv')
df_highest_grossers = pd.read_csv('Data/HighestGrossers.csv')
df_popular_type = pd.read_csv('Data/PopularCreativeTypes.csv')
df_top_dist = pd.read_csv('Data/TopDistributors.csv')
df_top_gen = pd.read_csv('Data/TopGenres.csv')
df_top_rat = pd.read_csv('Data/TopGrossingRatings.csv')
df_top_sour = pd.read_csv('Data/TopGrossingSources.csv')
df_top_meth = pd.read_csv('Data/TopProductionMethods.csv')
df_wide_release = pd.read_csv('Data/WideReleasesCount.csv')

#### Function

In [43]:
# Remove the sign
# Need to be modified with exception for type changing
def sign_elimination(df,*sign):
    for i in range(len(sign)):
        for j in df.columns:
            if df[j].dtypes == 'object':
                df[j] = df[j].str.replace(sign[i],'')

#### Data Cleaning

##### AnnualTicketSales.csv

In [52]:
# Checking the data types and see which columns needed to be modified such as empty column and dollar sign/ comma
df_annual_sales.dtypes

YEAR                                    int64
TICKETS SOLD                           object
TOTAL BOX OFFICE                       object
TOTAL INFLATION ADJUSTED BOX OFFICE    object
AVERAGE TICKET PRICE                   object
dtype: object

In [51]:
# Remove the empty column
df_annual_sales.drop('Unnamed: 5', axis = 1, inplace = True)

In [53]:
# Replace the comma and dollar sign between the number
sign_elimination(df_annual_sales, ',','$')

In [56]:
# All dollar sign and commas were removed
df_annual_sales.head()

,YEAR,TICKETS SOLD,TOTAL BOX OFFICE,TOTAL INFLATION ADJUSTED BOX OFFICE,AVERAGE TICKET PRICE
0,2021,423774881,3881777912,3881777912,9.16
1,2020,223638958,2048534616,2048534616,9.16
2,2019,1228541629,11253443955,11253444050,9.16
3,2018,1311536128,11948096650,12013670952,9.11
4,2017,1225639761,10993991460,11226860216,8.97


In [57]:
# Change the data to int or float
df_annual_sales['TICKETS SOLD'] = df_annual_sales['TICKETS SOLD'].astype(int)
df_annual_sales['TOTAL BOX OFFICE'] = df_annual_sales['TOTAL BOX OFFICE'].astype(int)
df_annual_sales['TOTAL INFLATION ADJUSTED BOX OFFICE'] = df_annual_sales['TOTAL INFLATION ADJUSTED BOX OFFICE'].astype(int)
df_annual_sales['AVERAGE TICKET PRICE'] = df_annual_sales['AVERAGE TICKET PRICE'].astype(float)

In [59]:
# All object variable was change and it can be used for further investigation
df_annual_sales.dtypes

YEAR                                     int64
TICKETS SOLD                             int64
TOTAL BOX OFFICE                         int64
TOTAL INFLATION ADJUSTED BOX OFFICE      int64
AVERAGE TICKET PRICE                   float64
dtype: object

##### HighestGrossers.csv

In [63]:
# Quick observation for the data set
df_highest_grossers.head()

,YEAR,MOVIE,GENRE,MPAA RATING,DISTRIBUTOR,TOTAL FOR YEAR,TOTAL IN 2019 DOLLARS,TICKETS SOLD
0,1995,Batman Forever,Drama,PG-13,Warner Bros.,184031112,387522978,42306002
1,1996,Independence Day,Adventure,PG-13,20th Century Fox,306169255,634504608,69269062
2,1997,Men in Black,Adventure,PG-13,Sony Pictures,250650052,500207943,54607854
3,1998,Titanic,Adventure,PG-13,Paramount Pictures,443319081,865842808,94524324
4,1999,Star Wars Ep. I: The Phantom Menace,Adventure,PG,20th Century Fox,430443350,776153749,84732942


In [67]:
# Check for the column type
df_highest_grossers.dtypes

YEAR                      int64
MOVIE                    object
GENRE                    object
MPAA RATING              object
DISTRIBUTOR              object
TOTAL FOR YEAR            int64
TOTAL IN 2019 DOLLARS     int64
TICKETS SOLD              int64
dtype: object

In [61]:
# Remove the comma and dollar sign for numeric values
# Make sure that the comma and dollar sign was not appeared in other column
sign_elimination(df_highest_grossers,',','$')

In [65]:
# Change the column type for numeric values
df_highest_grossers['TOTAL FOR YEAR'] = df_highest_grossers['TOTAL FOR YEAR'].astype(int)
df_highest_grossers['TOTAL IN 2019 DOLLARS'] = df_highest_grossers['TOTAL IN 2019 DOLLARS'].astype(int)
df_highest_grossers['TICKETS SOLD'] = df_highest_grossers['TICKETS SOLD'].astype(int)

In [66]:
# Make sure the column type was changed correctly
df_highest_grossers.dtypes

YEAR                      int64
MOVIE                    object
GENRE                    object
MPAA RATING              object
DISTRIBUTOR              object
TOTAL FOR YEAR            int64
TOTAL IN 2019 DOLLARS     int64
TICKETS SOLD              int64
dtype: object

In [68]:
df_highest_grossers.shape

(27, 8)

In [71]:
# Check the column with catagories
# Can be converted to function to prevent replicate workload
catagories_column = []
n = df_highest_grossers.shape[0]
for i in df_highest_grossers.columns:
    # catagories can be found if it less than the sample size
    if len(df_highest_grossers[i].unique()) < n: # Make sure it got replicate values
        catagories_column.append(i)
catagories_column # Column that needed to be encoded

['GENRE', 'MPAA RATING', 'DISTRIBUTOR']

In [ ]:
# Encoded

##### PopularCreativeTypes.csv

In [73]:
# Quick observation for the data set
df_popular_type

,RANK,CREATIVE TYPES,MOVIES,TOTAL GROSS,AVERAGE GROSS,MARKET SHARE
0,1.0,Contemporary Fiction,"7,442","$96,203,727,036","$12,927,133",40.46%
1,2.0,Kids Fiction,564,"$32,035,539,746","$56,800,602",13.47%
2,3.0,Science Fiction,724,"$29,922,660,857","$41,329,642",12.59%
3,4.0,Fantasy,759,"$21,724,062,575","$28,621,953",9.14%
4,5.0,Super Hero,129,"$20,273,157,911","$157,156,263",8.53%
5,6.0,Historical Fiction,"1,487","$18,521,260,744","$12,455,454",7.79%
6,7.0,Dramatization,"1,175","$15,715,191,699","$13,374,631",6.61%
7,8.0,Factual,"2,467","$2,960,327,207","$1,199,970",1.25%
8,9.0,Multiple Creative Types,42,"$117,574,526","$2,799,393",0.05%
9,NaN,NaN,NaN,NaN,NaN,NaN


In [80]:
# Remove the null row
df_popular_type.drop(9, axis = 0, inplace= True)

In [77]:
# Remove the commas ,dollar sign and percentage sign
sign_elimination(df_popular_type,',','$','%')

In [78]:
# Check the type for data set
df_popular_type.dtypes

RANK              float64
CREATIVE TYPES     object
MOVIES             object
TOTAL GROSS        object
AVERAGE GROSS      object
MARKET SHARE       object
dtype: object

In [81]:
# Change the type for certain column
# MARKET SHARE needed to changed to float and divide by 100
df_popular_type['MOVIES'] = df_popular_type['MOVIES'].astype(int)
df_popular_type['TOTAL GROSS'] = df_popular_type['TOTAL GROSS'].astype(int)
df_popular_type['AVERAGE GROSS'] = df_popular_type['AVERAGE GROSS'].astype(int)
df_popular_type['MARKET SHARE'] = df_popular_type['MARKET SHARE'].astype(float)
df_popular_type['MARKET SHARE'] = df_popular_type['MARKET SHARE'] / 100

In [82]:
# Make sure the column types was changed correctly
df_popular_type.dtypes

RANK              float64
CREATIVE TYPES     object
MOVIES              int64
TOTAL GROSS         int64
AVERAGE GROSS       int64
MARKET SHARE      float64
dtype: object

In [83]:
# Quick check for the data set
df_popular_type.head()

,RANK,CREATIVE TYPES,MOVIES,TOTAL GROSS,AVERAGE GROSS,MARKET SHARE
0,1.0,Contemporary Fiction,7442,96203727036,12927133,0.4046
1,2.0,Kids Fiction,564,32035539746,56800602,0.1347
2,3.0,Science Fiction,724,29922660857,41329642,0.1259
3,4.0,Fantasy,759,21724062575,28621953,0.0914
4,5.0,Super Hero,129,20273157911,157156263,0.0853
